In [0]:
!mkdir .kaggle
import json
token = {"username":"bgudltaldanny","key":"65b62f1a36c14c834b4e88037463a4f8"}
with open('.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [3]:
!cp .kaggle/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6


!kaggle competitions download -c home-depot-product-search-relevance
!unzip home-depot-product-search-relevance.zip -d home-depot-product-search-relevance/
!unzip home-depot-product-search-relevance/train.csv.zip -d home-depot-product-search-relevance/
!unzip home-depot-product-search-relevance/test.csv.zip -d home-depot-product-search-relevance/

Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.0.1)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl
 85% 59.0M/69.4M [00:00<00:00, 37.0MB/s]
100% 69.4M/69.4M [00:00<00:00, 77.5MB/s]
Archive:  home-depot-product-search-relevance.zip
  inflating: home-depot-product-search-relevance/attributes.csv.zip  
  inflating: home-depot-product-search-relevance/product_descriptions.csv.zip  
  inflating: home-depot-product-search-relevance/relevance_instructions.docx  
  inflating: home-depot-product-search-relevance/sample_submission.csv.zip  
  inflating: home-depot-product-search-relevance/test.csv.zip  
  inflating: home-depot-product-search-relevance/train.csv.zip  
Archive:  home-depot-product-search-relevance/train.csv.zip
  inflating: home-depot-product-search-relevance/tra

In [0]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf
from keras.utils import Sequence
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout,Conv1D, MaxPool1D,Concatenate, LSTM, Reshape, CuDNNLSTM
from keras.optimizers import RMSprop
import keras.backend as K
from sklearn.pipeline import Pipeline,FeatureUnion,make_pipeline
from sklearn.preprocessing import FunctionTransformer,KBinsDiscretizer,OneHotEncoder
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score,cross_val_predict,cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer,TfidfTransformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor


In [0]:
path='./home-depot-product-search-relevance'
train=pd.read_csv(path+'/train.csv',encoding="Latin-1")

In [35]:
train.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


In [0]:
max_product_title_len=train.product_title.map(lambda x: len(x)).max()
max_search_term=train.search_term.map(lambda x: len(x)).max()
max_input_len=max(max_product_title_len,max_search_term)

In [0]:
class MyCharGenerator(Sequence):
  def __init__(self,padding,df,batch_size=32,char_value=0):
    self.padding=padding
    self.char_value=char_value
    self.df=df
    self.batch_size=batch_size


  def __iter__(self):
    while(True):
      X1 = []
      X2 = []
      y = []

      for index,row in self.df.iterrows():
        y.append((row['relevance'] - 1) / 2)
        st_list= [ord(c) for c in list(row['search_term'])]
        pt_list=[ord(c) for c in list(row['product_title'])]
        #X1.append(np.pad(st_list,(0,self.padding-len(st_list))))
        #X2.append(np.pad(pt_list,(0,self.padding-len(pt_list))))
        X1.append(np.reshape(np.pad(st_list,(0,self.padding-len(st_list))),(147,1)))
        X2.append(np.reshape(np.pad(pt_list,(0,self.padding-len(pt_list))),(147,1)))
        # assume there's one document per line, tokens separated by whitespace
        if (index + 1) % self.batch_size == 0:
          y=np.array(y)

          yield (np.array([X1,X2]),y)
          X1 = []
          X2 = []
          y=[]

      y=np.array(y)
      if(len(y) > 0):
        yield np.array([X1,X2]),y


def new_loss(y_actual,y_predicted):
  delta=y_actual*0.15
  diff=K.abs(y_actual-y_predicted)
  ans = K.switch(K.less_equal(diff,delta), lambda: K.square(diff),lambda:1.0- diff)
  return ans

In [0]:
class MyWordGenerator(Sequence):
  def __init__(self,padding,df,batch_size=32,char_value=0):
    self.padding=padding
    self.char_value=char_value
    self.df=df
    self.batch_size=batch_size


  def __iter__(self):
    while(True):
      X1 = []
      X2 = []
      y = []

      for index,row in self.df.iterrows():
        y.append((row['relevance'] - 1) / 2)
        st_list= [ord(c) for c in row['search_term']]
        pt_list=[ord(c) for c in list(row['product_title'])]
        #X1.append(np.pad(st_list,(0,self.padding-len(st_list))))
        #X2.append(np.pad(pt_list,(0,self.padding-len(pt_list))))
        X1.append(np.reshape(np.pad(st_list,(0,self.padding-len(st_list))),(147,1)))
        X2.append(np.reshape(np.pad(pt_list,(0,self.padding-len(pt_list))),(147,1)))
        # assume there's one document per line, tokens separated by whitespace
        if (index + 1) % self.batch_size == 0:
          y=np.array(y)

          yield (np.array([X1,X2]),y)
          X1 = []
          X2 = []
          y=[]

      y=np.array(y)
      if(len(y) > 0):
        yield np.array([X1,X2]),y


def new_loss(y_actual,y_predicted):
  delta=y_actual*0.15
  diff=K.abs(y_actual-y_predicted)
  ans = K.switch(K.less_equal(diff,delta), lambda: K.square(diff),lambda:1.0- diff)
  return ans

In [0]:
myGen=MyCharGenerator(padding=147,df=train,batch_size=1)
it = iter(myGen)

X1 = []
X2 = []
Y = []
for time in range(len(train)):
  [X1_gen,X2_gen],y_gen = next(it)
  X1.append([X for X in X1_gen[0]])
  X2.append([X for X in X2_gen[0]])
  Y.append(y_gen[0])

X1_train,X1_test,X2_train,X2_test,y_train,y_test = train_test_split(X1,X2,Y,test_size=0.2)

X1_train = np.array(X1_train)
X2_train = np.array(X2_train)
y_train = np.array(y_train)
X1_test = np.array(X1_test)
X2_test = np.array(X2_test)
y_test = np.array(y_test)


In [0]:
def network(n_fetures):
  inp=Input(shape=(n_fetures,1))
  x = CuDNNLSTM(max_input_len)(inp)
  x = Reshape((max_input_len,1))(x)
  x=Conv1D(filters=32,kernel_size=10,activation='relu')(x)
  x=MaxPool1D()(x)
  x=Conv1D(filters=64,kernel_size=10,activation='relu')(x)
  x=MaxPool1D()(x)
  x=Conv1D(filters=128,kernel_size=10,activation='relu')(x)
  x=Flatten()(x)
  x=Dense(128,activation='relu')(x)
  model= Model(inp,x)

  search_term_input=Input(shape=(n_fetures,1),name='inp1')
  product_title_input=Input(shape=(n_fetures,1),name='inp2')

  search_term_encoded=model(search_term_input)
  product_title_encoded=model(product_title_input)


  x_connected=Concatenate()([search_term_encoded,product_title_encoded])
  x_connected=Dense(1,activation='sigmoid')(x_connected)

  connected_model=Model(inputs=[search_term_input,product_title_input],output=x_connected)
  

  return connected_model


In [0]:
siames_model=network(max_input_len)
siames_model.summary()
siames_model.compile(loss=new_loss ,optimizer='adam',metrics=['mae','accuracy'])

Model: "model_84"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp1 (InputLayer)               (None, 147, 1)       0                                            
__________________________________________________________________________________________________
inp2 (InputLayer)               (None, 147, 1)       0                                            
__________________________________________________________________________________________________
model_83 (Model)                (None, 128)          535336      inp1[0][0]                       
                                                                 inp2[0][0]                       
__________________________________________________________________________________________________
concatenate_38 (Concatenate)    (None, 256)          0           model_83[1][0]            

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


In [0]:
model_hist = siames_model.fit([X1_train,X2_train],y_train,verbose=2,epochs=3,batch_size=20,validation_data=([X1_test,X2_test],y_test))

Train on 59253 samples, validate on 14814 samples
Epoch 1/3
 - 69s - loss: 0.2817 - mean_absolute_error: 0.6899 - acc: 0.0282 - val_loss: 0.2796 - val_mean_absolute_error: 0.6908 - val_acc: 0.0296
Epoch 2/3
 - 65s - loss: 0.2810 - mean_absolute_error: 0.6908 - acc: 0.0281 - val_loss: 0.2796 - val_mean_absolute_error: 0.6908 - val_acc: 0.0296
Epoch 3/3
 - 65s - loss: 0.2810 - mean_absolute_error: 0.6908 - acc: 0.0281 - val_loss: 0.2796 - val_mean_absolute_error: 0.6908 - val_acc: 0.0296


In [0]:
Naive_X = []
Naive_Y = []
for index,row in train.iterrows():
  #print(self.counter)
  Naive_Y.append(row['relevance'])
  Naive_X.append(''.join(list(row['search_term'])) + ' ' +''.join(list(row['product_title'])))

X_naive_train,X_naive_test,y_naive_train,y_naive_test = train_test_split(Naive_X,Naive_Y,test_size=0.2)

pipeline = Pipeline([
    ('vect', CountVectorizer(stop_words=[' '], max_df=0.5, max_features=100,lowercase=False,analyzer='char')),
    #('tfidf', TfidfTransformer(use_idf=True, norm='l2')),
    ('clf', LinearRegression()),
])
naive_model = cross_validate(pipeline, X_naive_train, y_naive_train, cv=5,return_estimator=True)

predictions = naive_model['estimator'][4].predict(X_naive_test)
mae = np.absolute((np.array(y_naive_test)-np.array(predictions))).mean()
rmse = math.sqrt(((np.array(y_naive_test)-np.array(predictions))**2).mean())
print("MAE: %.3f" %mae)
print("RMSE: %.3f" %rmse)

MAE: 0.436
RMSE: 0.533


In [0]:


siames_model_ML=network(max_input_len)
siames_model_ML.layers.pop(-1)
siames_model_ML.summary()
siames_model_ML.compile(loss=new_loss ,optimizer='SGD',metrics=['mae','accuracy'])
model_hist_ml = siames_model_ML.fit([X1_train,X2_train],y_train,verbose=2,epochs=10,batch_size=20,validation_data=([X1_test,X2_test],y_test))

ml_predictions_train = siames_model_ML.predict([X1_train,X2_train])
ml_predictions_test = siames_model_ML.predict([X1_test,X2_test])

ml_knn_model = KNeighborsRegressor()
ml_knn_model.fit(ml_predictions_train,y_train)
ml_knn_predictions = ml_knn_model.predict(ml_predictions_test)

ml_rf_model = RandomForestRegressor()
ml_rf_model.fit(ml_predictions_train,y_train)
ml_rf_predictions = ml_rf_model.predict(ml_predictions_test)


mae_knn = np.absolute((np.array(y_naive_test)-np.array(ml_knn_predictions))).mean()
rmse_knn = math.sqrt(((np.array(y_naive_test)-np.array(ml_knn_predictions))**2).mean())

mae_rf = np.absolute((np.array(y_naive_test)-np.array(ml_rf_predictions))).mean()
rmse_rf = math.sqrt(((np.array(y_naive_test)-np.array(ml_rf_predictions))**2).mean())

print("KNN MAE: %.3f" %mae_knn)
print("KNN RMSE: %.3f" %rmse_knn)

print("RANDOM MAE: %.3f" %mae_rf)
print("RANDOM RMSE: %.3f" %rmse_rf)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Model: "model_80"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp1 (InputLayer)               (None, 147, 1)       0                                            
__________________________________________________________________________________________________
inp2 (InputLayer)               (None, 147, 1)       0                                            
__________________________________________________________________________________________________
model_79 (Model)                (None, 128)          558428      inp1[0][0]                       
                                                                 inp2[0][0]                       
__________________________________________________________________________________________________
concatenate_36 (Concatenate)    (None, 256)          0           model_79[1][0]            

We explored the data and decided we need to do some preprocessing

we found some examples for words that ment the same thing but were wrote differently.

for example in the 'product title' column there is no 'inches' there is 'in.'
but in the 'search term' there is 'inches'

In [37]:
 print ('\'product_titles\' contains \'inches\'')
 display(train.loc[train['product_title'].str.contains(' inches ')])
 print ('\n\n\n\'search_terms\' contains \'inches\'')
 display(train.loc[train['search_term'].str.contains(' inches ')].head(5))

'product_titles' contains 'inches'


,id,product_uid,product_title,search_term,relevance





'search_terms' contains 'inches'


,id,product_uid,product_title,search_term,relevance
4015,12507,102193,Samsung 28.15 cu. ft. 4-Door French Door Refri...,33 inches wide samsung,2.67
13646,42206,110179,JELD-WEN Smooth 4-Panel Primed Molded Interior...,interior doors 82 inches in length,2.33
14917,46069,111389,Paslode 3 in. x 0.131-Gauge 30å¡ Brite Smooth ...,3 .5 inches paper tape collated,2.00
15609,48121,112069,Roberts Laminate Cutter for Cross Cutting up t...,metric to inches converter,1.67
17099,52792,113648,Masonite 30 in. x 80 in. Textured 6-Panel Holl...,interior doors 82 inches in length,2.67


In [0]:
import re
def preprocess_sentences(s):
  s = s.lower()
  s = re.sub(r"(\d+)'",r'\1in.',s)
  s = s.replace("inches","in.") 
  s = s.replace("inch","in.")
  s = s.replace(" in ","in. ") 
  s = s.replace(" in.","in.") 

  s = s.replace("''","ft.") 
  s = s.replace(" feet ","ft. ") 
  s = s.replace("feet","ft.") 
  s = s.replace("foot","ft.") 
  s = s.replace(" ft ","ft. ") 
  s = s.replace(" ft.","ft.") 

  s = s.replace(" pounds ","lb. ")
  s = s.replace(" pound ","lb. ") 
  s = s.replace("pound","lb.") 
  s = s.replace(" lb ","lb. ") 
  s = s.replace(" lb.","lb.") 
  s = s.replace(" lbs ","lb. ") 
  s = s.replace("lbs.","lb.") 

  s = s.replace(" x "," xby ")
  s = s.replace("*"," xby ")
  s = s.replace(" by "," xby")
  s = s.replace("x0"," xby 0")
  s = s.replace("x1"," xby 1")
  s = s.replace("x2"," xby 2")
  s = s.replace("x3"," xby 3")
  s = s.replace("x4"," xby 4")
  s = s.replace("x5"," xby 5")
  s = s.replace("x6"," xby 6")
  s = s.replace("x7"," xby 7")
  s = s.replace("x8"," xby 8")
  s = s.replace("x9"," xby 9")
  s = s.replace("0x","0 xby ")
  s = s.replace("1x","1 xby ")
  s = s.replace("2x","2 xby ")
  s = s.replace("3x","3 xby ")
  s = s.replace("4x","4 xby ")
  s = s.replace("5x","5 xby ")
  s = s.replace("6x","6 xby ")
  s = s.replace("7x","7 xby ")
  s = s.replace("8x","8 xby ")
  s = s.replace("9x","9 xby ")

  s = s.replace(" sq ft","sq.ft. ") 
  s = s.replace("sq ft","sq.ft. ")
  s = s.replace("sqft","sq.ft. ")
  s = s.replace(" sqft ","sq.ft. ") 
  s = s.replace("sq. ft","sq.ft. ") 
  s = s.replace("sq ft.","sq.ft. ") 
  s = s.replace("sq feet","sq.ft. ") 
  s = s.replace("square feet","sq.ft. ") 

  s = s.replace(" gallons ","gal. ") 
  s = s.replace(" gallon ","gal. ") 
  s = s.replace("gallons","gal.") 
  s = s.replace("gallon","gal.") 
  s = s.replace(" gal ","gal. ") 
  s = s.replace(" gal","gal.") 

  s = s.replace("ounces","oz.")
  s = s.replace("ounce","oz.")
  s = s.replace(" oz.","oz. ")
  s = s.replace(" oz ","oz. ")

  s = s.replace("centimeters","cm.")    
  s = s.replace(" cm.","cm.")
  s = s.replace(" cm ","cm. ")
  
  s = s.replace("milimeters","mm.")
  s = s.replace(" mm.","mm.")
  s = s.replace(" mm ","mm. ")
  
  s = s.replace("°","deg. ")
  s = s.replace("degrees","deg. ")
  s = s.replace("degree","deg. ")
  
  s = s.replace("volts","volt. ")
  s = s.replace("volt","volt. ")

  s = s.replace("watts","watt. ")
  s = s.replace("watt","watt. ")

  return s

In [55]:
re.sub(r"(\d+)'",r'\1in.',"he4yy3' 4'")

'he4yy3in. 4in.'

In [42]:
processed_train = train.copy()
processed_train['search_term'] = processed_train['search_term'].map(lambda x:preprocess_sentences(x))
processed_train['product_title'] = processed_train['product_title'].map(lambda x:preprocess_sentences(x))

print ('\'product_titles\' contains \'inches\'')
display(processed_train.loc[processed_train['product_title'].str.contains(' in. ')])
print ('\n\n\n\'search_terms\' contains \'inches\'')
display(processed_train.loc[processed_train['search_term'].str.contains(' in. ')].head(5))

'product_titles' contains 'inches'


,id,product_uid,product_title,search_term,relevance
18498,57112,115108,ready-strip 32oz. environmentally friendly dr...,adhesive slide strip,1.67
25119,77115,122211,bonded logic inc ultrasonic 12in. xby 12in. ac...,12in. ceiling tile,2.00
25120,77121,122211,bonded logic inc ultrasonic 12in. xby 12in. ac...,sound dampening,2.67
25121,77123,122211,bonded logic inc ultrasonic 12in. xby 12in. ac...,sounds panels,2.00
67581,202995,191353,mohawk home ink swirl cocoa 2ft. xby 8ft. runner,rug ruunners,2.00





'search_terms' contains 'inches'


,id,product_uid,product_title,search_term,relevance
42161,128340,144629,liberty 20in. european self-closing drawer sli...,18.75 in. drawer slides,2.67
54368,164835,164764,kaleen matira blue 2ft. xby 3ft. indoor/outdoo...,"kaleen rugs, inc matira area rug 3 x",2.00
55418,167918,166665,kaleen matira blue 2ft. xby 3ft. indoor/outdoo...,"kaleen rugs, inc matira area rug 3 x",3.00
60397,182300,176124,kaleen matira blue 3ft. xby 5ft. indoor/outdoo...,"kaleen rugs, inc matira area rug 3 x",3.00
